# Deep Q-Network (DQN)
---
In this notebook, you will implement a DQN agent with OpenAI Gym's LunarLander-v2 environment.

### 1. Import the Necessary Packages

In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
import tqdm as tqdm
%matplotlib inline

### 2. Instantiate the Environment and Agent

Initialize the environment in the code cell below.

In [2]:
env = gym.make('LunarLander-v2', render_mode='rgb_array')
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)

State shape:  (8,)
Number of actions:  4


Before running the next code cell, familiarize yourself with the code in **Step 2** and **Step 3** of this notebook, along with the code in `dqn_agent.py` and `model.py`.  Once you have an understanding of how the different files work together, 
- Define a neural network architecture in `model.py` that maps states to action values.  This file is mostly empty - it's up to you to define your own deep Q-network!
- Finish the `learn` method in the `Agent` class in `dqn_agent.py`.  The sampled batch of experience tuples is already provided for you; you need only use the local and target Q-networks to compute the loss, before taking a step towards minimizing the loss.

Once you have completed the code in `dqn_agent.py` and `model.py`, run the code cell below.  (_If you end up needing to make multiple changes and get unexpected behavior, please restart the kernel and run the cells from the beginning of the notebook!_)

You can find the solution files, along with saved model weights for a trained agent, in the `solution/` folder.  (_Note that there are many ways to solve this exercise, and the "solution" is just one way of approaching the problem, to yield a trained agent._)

In [3]:
from dqn_agent import Agent
import matplotlib.animation
from IPython.display import HTML



def render(env, agent):
    plt.style.use('ggplot')

    state = env.reset(seed=505)[0]
    score = 0
    frames = []
    while True:
        out = env.render()
        frames.append(out)
        action = agent.act(state)
        state, reward, terminated, truncated, _ = env.step(action)
        score += reward
        if terminated or truncated:
            break

    fig = plt.figure()
    plt.axis('off')
    plt.grid(False)
    im = plt.imshow(out)

    def animation(i):
        im.set_data(frames[i])
        return im


    animation = matplotlib.animation.FuncAnimation(fig, animation, frames=len(frames), interval=50)
    plt.close()

    print('Final score:', score)
    env.close()
    return animation.to_html5_video()

agent = Agent(state_size=env.observation_space.shape[0], action_size=env.action_space.n, seed=0)
HTML(render(env, agent))

/home/110359@TRI.LAN/.local/lib/python3.10/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Final score: -107.96898188899681


### 3. Train the Agent with DQN

Run the code cell below to train the agent from scratch.  You are welcome to amend the supplied values of the parameters in the function, to try to see if you can get better performance!

In [4]:
%tensorboard --logdir=runs


UsageError: Line magic function `%tensorboard` not found.


In [5]:

def dqn(n_episodes=3000, max_t=500, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.

    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    best_score = 200
    bar = tqdm.tqdm(range(1, n_episodes+1))
    for i_episode in bar:
        state = env.reset()[0]
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, d1, d2, _ = env.step(action)
            score += reward
            done = d1 or d2
            agent.step(state, action, reward, next_state, done)
            state = next_state
            if done:
                break
        agent.report_score(score)
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        mean_score = np.mean(scores_window)
        bar.set_postfix(mean_score=mean_score, eps=eps)
        if i_episode % 100 == 0:
            if mean_score > best_score:
                best_score = mean_score
                torch.save(agent.qnetwork_local.state_dict(), f'checkpoint_{i_episode}_{mean_score:.2f}.pth')

    return scores


scores = dqn()
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

  2%|▏         | 49/3000 [00:04<04:45, 10.32it/s, eps=0.782, mean_score=-173]


KeyboardInterrupt: 

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [6]:
# load the weights from file
agent.qnetwork_local.load_state_dict(torch.load('checkpoint.pth'))

HTML(render(env, agent))

Final score: 244.4377383467735


### 5. Explore

In this exercise, you have implemented a DQN agent and demonstrated how to use it to solve an OpenAI Gym environment.  To continue your learning, you are encouraged to complete any (or all!) of the following tasks:
- Amend the various hyperparameters and network architecture to see if you can get your agent to solve the environment faster.  Once you build intuition for the hyperparameters that work well with this environment, try solving a different OpenAI Gym task with discrete actions!
- You may like to implement some improvements such as prioritized experience replay, Double DQN, or Dueling DQN! 
- Write a blog post explaining the intuition behind the DQN algorithm and demonstrating how to use it to solve an RL environment of your choosing.  